El siguiente script muestra la construcción de una Red multicapa, podiendo definir el numero de entradas, pesos, capas y neuronas por capas.

Se construye apartir de 3 pasos (Creando 3 clases)

    1.Clase Neuron : Define el numero de entradas y pesos aleatorios
    
    2.Clase NeuronLayer : Numero de Neuronas por capas
    
    3.Clase NeuronalNetwork : Trabaja con las 2 clases anteriores e implenta la función de activacion hacia                   delante
    
Nota; Hace falta colocar la optimizacion y entrenamiento de la Red.

In [1]:
import random

BIAS = -1

##############
#Clase Neuron#
##############

#Clase que define el numero de entradas y los pesos para cada entrada

class Neuron:
    #Metodo constructor inicial
    def __init__(self, n_inputs ): #Colocamos un numero n de entradas (atributos)
        self.n_inputs = n_inputs
        random.seed(1)
        self.set_weights( [random.uniform(0,1) for x in range(0,n_inputs)] ) # Calculamos los pesos de las entradas de forma aleatoria

    #Metodo suma
    def sum(self, inputs ):
        # No incluimos el sesgo
        
        return sum(val*self.weights[i] for i,val in enumerate(inputs)) #Multiplicación vectorial y suma de las entradas con los pesos

    def set_weights(self, weights ):
        
        self.weights = weights
        
    def __str__(self):
        
        return 'Weights: %s, input: %s' % ( str(self.weights[:]),str(self.n_inputs) )


In [9]:
#Implementación de la primera clase Neuron, para definir el numero de entradas y los pesos
a = Neuron(3) #Numero de entradas
#Atributos
a.n_inputs #Numero de entradas definidas
a.weights #Pesos definidos de forma aleatoria para cada entrada
#Metodos
a.set_weights([1,2,2]) #Cambiar los pesos de las neuronas (En caso de que no querramos de forma aleatoria, debe preferencia que tenga la misma longitud que el numero de entradas)
a.weights
a.sum([3,1,3]) #Valores de cada entrada (el array colocado debe de ser de la misma longitud que las neuronas de entrada)
#Metodo __str__
print(a)

Weights: [1, 2, 2], input: 3


In [10]:
############################################
#Clase para el numero de neuronas por capas#
############################################

class NeuronLayer:
    def __init__(self, n_neurons, n_inputs):
        self.n_neurons = n_neurons
        self.neurons = [Neuron( n_inputs ) for _ in range(0,self.n_neurons)] #Numero de neuronas

    def __str__(self):
        return 'Layer:\n\t'+'\n\t'.join([str(neuron) for neuron in self.neurons])+''

In [12]:
#Ejemplo: Red de 3 capas con 2 entradas con pesos aleatorios        
b = NeuronLayer(3,6)
b.n_neurons
print(b)


Layer:
	Weights: [0.13436424411240122, 0.8474337369372327, 0.763774618976614, 0.2550690257394217, 0.49543508709194095, 0.4494910647887381], input: 6
	Weights: [0.13436424411240122, 0.8474337369372327, 0.763774618976614, 0.2550690257394217, 0.49543508709194095, 0.4494910647887381], input: 6
	Weights: [0.13436424411240122, 0.8474337369372327, 0.763774618976614, 0.2550690257394217, 0.49543508709194095, 0.4494910647887381], input: 6


In [14]:
###################################
#Se crea una clase NeuronalNetwork#
###################################

#Ocupando las 2 clases anteriores para crear el numero de entradas, pesos y numero de neuronas de capas

class NeuralNetwork:
    def __init__(self, n_inputs, n_outputs, n_neurons_to_hl, n_hidden_layers):
        #Atributos de la Red Neuronal (Variables de entrada, Variables de salida, Capas ocultas, Neuronas en cada capa, Pesos de las variables)
        self.n_inputs = n_inputs #n Entradas
        self.n_outputs = n_outputs #n Salidas
        self.n_hidden_layers = n_hidden_layers #n Capas ocultas
        self.n_neurons_to_hl = n_neurons_to_hl #n Neuronoas para capas ocultas
        
        # Do not touch
        self._create_network()
        self._n_weights = None #Pesos 
        # end
    #Metodo para crear la Red ocupando la case NeuronLayer (Define el numero de capas ocultas y entradas)
    def _create_network(self):
        if self.n_hidden_layers>0:
            # Creacion de la primera capa
           self.layers = [NeuronLayer( self.n_neurons_to_hl,self.n_inputs )]
           
           # Creacion de las capas ocultas
           self.layers += [NeuronLayer( self.n_neurons_to_hl,self.n_neurons_to_hl ) for _ in range(0,self.n_hidden_layers)]
           
           # Capa oculta salida
           
           self.layers += [NeuronLayer( self.n_outputs,self.n_neurons_to_hl )]
           
        else:
            # If we don't require hidden layers
           self.layers = [NeuronLayer( self.n_outputs,self.n_inputs )]
    
    #Metodo para obtener los pesos
    def get_weights(self):
        
        weights = []
        
        for layer in self.layers:
            
            for neuron in layer.neurons:
                
                weights += neuron.weights
                
        return weights

    @property
    
    def n_weights(self):
        
        if not self._n_weights:
            
            self._n_weights = 0
            
            for layer in self.layers:
                
                for neuron in layer.neurons:
                    
                    self._n_weights += neuron.n_inputs+1 # +1 for bias weight
                    
        return self._n_weights

    def set_weights(self, weights ):
        
        assert len(weights)==self.n_weights, "Incorrect amount of weights."
        
        stop = 0
        
        for layer in self.layers:
            
            for neuron in layer.neurons:
                start, stop = stop, stop+(neuron.n_inputs+1)
                neuron.set_weights( weights[start:stop] )
                
        return self

    def update(self, inputs ):
        
        assert len(inputs)==self.n_inputs, "Incorrect amount of inputs."

        for layer in self.layers:
            
            outputs = []
            
            for neuron in layer.neurons:
                tot = neuron.sum(inputs) + neuron.weights[-1]*BIAS
                outputs.append( self.relu(tot) )
            inputs = outputs
        return outputs

    def relu(self, activation):
        
        if activation <= 0:
            output = -1
        elif activation > 0:
            output = 1
            
        # Return the value just calculated
        
        return(output)

    def __str__(self):
        
        return '\n'.join([str(i+1)+' '+str(layer) for i,layer in enumerate(self.layers)])

In [16]:
#Definicion de la Red Neuronal            
Red = NeuralNetwork(2,1,2,1)

print(Red)

1 Layer:
	Weights: [0.13436424411240122, 0.8474337369372327], input: 2
	Weights: [0.13436424411240122, 0.8474337369372327], input: 2
2 Layer:
	Weights: [0.13436424411240122, 0.8474337369372327], input: 2
	Weights: [0.13436424411240122, 0.8474337369372327], input: 2
3 Layer:
	Weights: [0.13436424411240122, 0.8474337369372327], input: 2


In [22]:
#Atributos
Red.layers
Red.n_hidden_layers
Red.n_inputs
Red.n_neurons_to_hl
Red.n_outputs
Red.n_weights

15

In [23]:
#Funcion de activacion
Red.relu(-1)

-1

In [24]:
#Resultado final
sum(Red.update([2,2]))

1